<a href="https://colab.research.google.com/github/yuyehara/IS-GEO2022_Angeve/blob/main/clippingRaster_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install geopandas
!pip install earthpy
!pip install rasterio
!pip install rioxarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 6.7 MB/s 
     |████████████████████████████████| 6.3 MB 35.7 MB/s 
     |████████████████████████████████| 16.7 MB 41.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 7.1 MB/s 
     |████████████████████████████████| 19.3 MB 8.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 3.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for rioxarray: filename=rioxarray-0.9.1-py3-none-any.whl size=54611 sha256=291bf63148a455cd9e1a94390e88329ee16c2db752

# New Section

In [3]:
import os
import rioxarray as rxr
import rasterio
import geopandas as gpd


In [4]:
# Original code ref.: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/crop-raster-data-with-shapefile-in-python/
# import libraries
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# from shapely.geometry import mapping
# import rasterio
# import geopandas as gpd0
0# import earthpy as et
# import earthpy.plot as ep

0

In [5]:
# set a destination folder path
outFolderPath = "/content/drive/MyDrive/IS-GEO-22/croppedImages"
os.chdir(outFolderPath)

In [6]:
# get vector using geopanda
palm_0339_poly = gpd.read_file("/content/drive/MyDrive/IS-GEO-22/palm_0339_2.shp")

# get a raster
raster_folder ="/content/drive/MyDrive/IS-GEO-22" 
palm_0339_image = "img_0339.tif"
palm_0339_fn = os.path.join(raster_folder, palm_0339_image)
palm_0339_ras = rasterio.open(palm_0339_fn)

In [7]:
import fiona
from pyproj import Proj, transform
polyFile = fiona.open('/content/drive/MyDrive/IS-GEO-22/palm_0339_2.shp')
crs_poly = Proj(polyFile.crs)
print(crs_poly)

palm_0339_poly.Id = range(palm_0339_poly.shape[0])

proj=utm zone=4 datum=WGS84 units=m no_defs ellps=WGS84 towgs84=0,0,0


In [8]:
# check the coodinate reference system (crs)
# project vector on the raster layer because the all raster layers have the same spatial reference
# the traingin data set are derived from the raster layers

polyCrs = palm_0339_poly.crs
rasCrs = palm_0339_ras.crs

palm_0339_poly.to_crs(rasCrs, inplace = True)
print(palm_0339_poly.crs)

print("palm_0339_poly =", polyCrs)
print("palm_0339_ras =", rasCrs)

PROJCS["WGS_1984_UTM_Zone_4N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-159],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32604"]]
palm_0339_poly = PROJCRS["WGS 84 / UTM zone 4N",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]],ID["EPSG",6326]],PRIMEM["Greenwich",0,ANGLEUNIT["Degree",0.0174532925199433]]],CONVERSION["UTM zone 4N",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["

In [9]:
polyCrs

<Projected CRS: PROJCRS["WGS 84 / UTM zone 4N",BASEGEOGCRS["WGS 84 ...>
Name: WGS 84 / UTM zone 4N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
- h[up]: Ellipsoidal height (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: UTM zone 4N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:

rasCrs

CRS.from_epsg(32604)

In [11]:
out_meta = palm_0339_ras.meta

In [12]:
def add_a_to_b(a, b):
  return a + b

In [13]:
def rasterio_write(input_raster_folder, input_raster_image, shape, output_filename, output_Folder = None):
  ''' This function takes in a raster image, and clips individual polygons within that image into their own individual rasters.
  input_raster_folder: the path to the folder where the raster is located
  input_raster_image: the specific raster file to analyze
  '''
  raster_filepath = os.path.join(input_raster_folder, input_raster_image)
  with rasterio.open(raster_filepath) as src:
    out_image, out_transform = rasterio.mask.mask(src, shape, crop=True)
    out_meta = src.meta

  out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
  
  if output_Folder:
    out_path = os.path.join(output_Folder, output_filename)
  else:
    out_path = os.path.join(input_raster_folder, output_filename)

  with rasterio.open(out_path, "w", **out_meta) as dest:
    dest.write(out_image)

  return out_image, out_meta # new data





In [14]:
outFolderPath = "croppedImages"

In [16]:
# create a new file by opening up a raster file, then crop the raster by each polygon and name each one
# input parameters for my function
input_raster_folder = "/content/drive/MyDrive/IS-GEO-22"  
input_raster_image = "img_0339.tif"
output_Folder = "/content/drive/MyDrive/IS-GEO-22/croppedImages"  

# Run function
new_fn = palm_0339_poly[['Id','geometry']].apply(lambda x: rasterio_write(input_raster_folder, input_raster_image, [x.geometry], input_raster_image+ str(x.Id) + ".tif", output_Folder), axis = 1)
# new_fn = palm_0339_poly[['Id','geometry']].apply(lambda polygon: rasterio_write(palm_0339_fn, palm_0339_fn + str(polygon.Id) + ".tif", [polygon.geometry], outFolderPath), axis = 1)
